In [45]:
import os
import re
import requests
import pandas as pd
from io import StringIO

from bs4 import BeautifulSoup
import time
from lxml import html

In [16]:
actasComisiones = "https://www2.congreso.gob.pe/Sicr/ApoyComisiones/comision2011.nsf/03pa_actas2305"


In [28]:
base_url = actasComisiones

response = requests.get(base_url)
tree = html.fromstring(response.text)
link_element = tree.xpath("/html/body/form/p/table[2]/tbody/tr[10]/td[4]/font/a[2]")
if link_element:
    # Get the href attribute of the <a> element (the link)
    link = link_element[0].get("href")
    print(link)
else:
    print("The <a> element was not found on the page.")



The <a> element was not found on the page.


In [15]:

response.raise_for_status()
soup = BeautifulSoup(response.content, "html.parser")
pdf_links = soup.find_all("a", href=lambda href: href and href.endswith(".pdf") and ("SALUD" in href or "Salud" in href or "salud" in href))


In [13]:
# Importamos las librerías que utilizaremos
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By

In [29]:
# Scrappeo de la web del OSCE
# Definimos algunas opciones para abrir la página
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)


# Abrimos la página
driver.maximize_window()

driver.get(actasComisiones)

perParl = 2 #Va de 2 a 4
click_1 = driver.find_element( By.XPATH, f"/html/body/form/p/table[2]/tbody/tr[{perParl}]/td[1]/table/tbody/tr/td[1]/a/img")
click_1.click()
time.sleep(3)

perAnual = 3 # va de perParl+1 hasta perParl+5 como máximo
click_2 = driver.find_element( By.XPATH, f"/html/body/form/p/table[2]/tbody/tr[{perAnual}]/td[1]/table/tbody/tr/td[1]/a/img")
click_2.click()
time.sleep(3)

if perParl == 2:
    comOrd = perAnual + 4
    click_3 = driver.find_element( By.XPATH, f"/html/body/form/p/table[2]/tbody/tr[{comOrd}]/td[2]/table/tbody/tr/td[1]/a/img")
    click_3.click()
    time.sleep(5)
    for i in range(1, 2):
        comision = comOrd + i
        click_4 = driver.find_element( By.XPATH, f"/html/body/form/p/table[2]/tbody/tr[{comision}]/td[2]/table/tbody/tr/td[1]/a/img")
        click_4.click()
        time.sleep(3)
        #DESCARGAR AQUÍ LA INFORMACIÓN
        for n in range(1,2):
            current_url = driver.current_url
            response = requests.get(current_url)
            tree = html.fromstring(response.text)
            n_link = comision + n
            link_element = tree.xpath("/html/body/form/p/table[2]/tbody/tr[{n_link}]/td[4]/font/a[2]")
            link = link_element[0].get("href")
        click_4b = driver.find_element( By.XPATH, f"/html/body/form/p/table[2]/tbody/tr[{comision}]/td[2]/table/tbody/tr/td[2]/a/img")
        click_4b.click()
else:
    comOrd = perAnual + 1 #range(1,24)
    click_3 = driver.find_element( By.XPATH, f"/html/body/form/p/table[2]/tbody/tr[{comOrd}]/td[1]/table/tbody/tr/td[1]/a/img")
    click_3.click()
    time.sleep(3)
    
# for range(1,24)

#driver.find_element()

XPathEvalError: Invalid expression

# Integrantes de grupos parlamentarios
Scrappeo de la web del congreso para construir una base de datos con la información de todos los congresistas de los periodos parlamentarios desde 1992 hasta el 2026

In [35]:
keyList = ['CCD 1992-1995', '1995-2000','2000-2001','2001-2006','2006-2011','2011-2016','2016-2021','2021-2026']

linkList = ['https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=1','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=2','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=3','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=4','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=5','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=6','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=7','https://www.congreso.gob.pe/pleno/congresistas/?=undefined&m1_idP=13']

for i, key in enumerate(keyList):
    url = linkList[i]
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36', }
    response = requests.get(url, headers = headers)
    html_io = StringIO(response.text)
    df = pd.read_html(html_io)
    df = df[0]
    df['Periodo'] = key
    if i == 0 :
        dfCongresistas = df
    else:
        dfCongresistas = pd.concat([dfCongresistas, df])

dfCongresistas = dfCongresistas.iloc[:,1:]

In [57]:
# Your HTML content
def extraeInfoCongresista(textHtml):
    html_content = textHtml

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Initialize an empty dictionary to store the key-value pairs
    info_dict = {}

    # Find all spans with class 'field' and their corresponding values
    field_spans = soup.find_all('span', class_='field')
    for field_span in field_spans:
        key = field_span.get_text(strip=True)
        value_span = field_span.find_next_sibling('span')
        value = value_span.get_text(strip=True) if value_span else None
        info_dict[key] = value

    # List of keys you want to select
    selected_keys = ["Nombres:", "Votación Obtenida:", "Inicio:", "Término:", "Grupo o Partido Político:", "Bancada:", "Representa a:", "Condición:"]

    # Create a new dictionary with only the selected keys
    selected_info = {key: info_dict[key] for key in selected_keys}

    return selected_info

In [65]:
congList = []

for id in range(1,1166):
    link = 'https://www.congreso.gob.pe/pleno/congresistas/?K=290&TPL=if0&a=iv&id=' + str(id)
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36', }
    response = requests.get(link, headers = headers)
    
    infoCong = extraeInfoCongresista(response.text)
    congList.append(infoCong)
    
dfCongresistas2 = pd.DataFrame(congList)

In [70]:
dfCongresistas2 = dfCongresistas2.reset_index()
dfCongresistas2['id'] = dfCongresistas2['index'] + 1
dfCongresistas2 = dfCongresistas2.iloc[:,1:]

# Votaciones de congresistas en el Pleno

In [164]:
def extraeListaLinks(textHtml):
    html_content = textHtml
    soup = BeautifulSoup(html_content, 'html.parser')
    href_tags = soup.find_all("a", href=lambda href: href  and ("vot" in href or "VOT" in href or "Vot" in href) and ("Apleno" in href))

    pdf_list = []
    for a in href_tags:
        link = a.attrs['href']
        pdf_list.append(link)
    
    for i, link in enumerate(pdf_list):
        match = re.search(r"javascript:openWindow\('(.+?)'\)", link)
        pdf_list[i]= match.group(1)
        
            
    return pdf_list

In [165]:
urlVotaciones = "https://www2.congreso.gob.pe/Sicr/RelatAgenda/PlenoComiPerm20112016.nsf/new_asistenciavotacion?OpenForm&ExpandView"

url = urlVotaciones
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36', }
response = requests.get(url, headers = headers)

text = response.text
linkList = extraeListaLinks(textHtml = text)



## Descarga de todos los pdf o Uso de los PDF con open()

In [132]:
path = r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\12. Congreso'
download_dir = path + "\pdf_votaciones_pleno"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

base_url = 'https://www2.congreso.gob.pe/Sicr/RelatAgenda/PlenoComiPerm20112016.nsf/'
downloaded_files = set()

for link in linkList:
    pdf_url= base_url + link
    pdf_name = pdf_url.split("/")[-1]

    if pdf_name in downloaded_files:
        continue

    pdf_path = os.path.join(download_dir, pdf_name)
    response = requests.get(pdf_url)

    with open(pdf_path, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded {pdf_name}")
    downloaded_files.add(pdf_name)



Downloaded Asis_vot_PROVISIONAL_matinal_05-10-23.pdf
Downloaded Asis_vot_PROVISIONAL_vespertina_05-10-23.pdf
Downloaded Asis_PROVISIONAL_05-10-23.pdf
Downloaded Asis_vot_PROVISIONAL_4-10-23.pdf
Downloaded Asis_vot_OFICIAL_21-09-23.pdf
Downloaded Asis_OFICIAL_sesión_solemne_20-09-23.pdf


KeyboardInterrupt: 

# Debates del pleno del congreso

In [162]:
def extraeListaLinks(textHtml):
    html_content = textHtml
    soup = BeautifulSoup(html_content, 'html.parser')
    href_tags = soup.find_all("a", href=lambda href: href  and "SesionesPleno" in href)

    pdf_list = []
    for a in href_tags:
        link = a.attrs['href']
        pdf_list.append(link)

    for i, link in enumerate(pdf_list):
        match = re.search(r"javascript:mapCat\('(.+?)'\)", link)
        pdf_list[i]= match.group(1)
        
    return pdf_list

In [163]:
urlPleno = 'https://www2.congreso.gob.pe/Sicr/DiarioDebates/Publicad.nsf/new_sesionespleno?OpenForm&ExpandView'

url = urlPleno
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36', }
response = requests.get(url, headers = headers)

text = response.text
linkList = extraeListaLinks(textHtml = text)
